## Download arXiv dataset regarding Errors-in-Variables models

In [2]:
import json
import os

import arxiv
import feedparser
import fitz
import requests

In [34]:
# Define the exact phrase to search
exact_phrase = "Errors-in-variables model"
output_directory = "/data/arxiv_stat_pdfs"  # Directory to save PDFs

# Construct the query URL for exact matches
query_url = f"http://export.arxiv.org/api/query?search_query=all:%22errors+in+variables+model%22&max_results=50"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Fetch results from the arXiv API
print(f"Searching arXiv for: {exact_phrase}")
response = requests.get(query_url)
if response.status_code != 200:
    print(f"Failed to fetch results: {response.status_code}")
    exit()

# Parse the results
feed = feedparser.parse(response.content)
entries = feed.entries

print(f"Found {len(entries)} papers matching the exact phrase.")

# Download PDFs
for entry in entries:
    try:
        title = entry.title
        pdf_url = entry.links[1].href  # The second link is typically the PDF
        pdf_path = os.path.join(output_directory, f"{entry.id.split('/')[-1]}.pdf")
        pdf_response = requests.get(pdf_url)
        with open(pdf_path, 'wb') as pdf_file:
            pdf_file.write(pdf_response.content)

    except Exception as e:
        print(f"Failed to download {title}: {e}")

# Print the number of PDFs downloaded
print("Finished downloading PDFs")


Searching arXiv for: Errors-in-variables model
Found 45 papers matching the exact phrase.
Finished downloading PDFs


## Extract Text from PDFs

extract text from the pdfs - should this be saved as a json or txt?
also run on hpc this is a lot

In [35]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    text = ""

    # Extract text from each page
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    return text

# extract text from all pdfs and save each in a json file
pdf_dir = "/data/arxiv_stat_pdfs"
text_dir = "/data/arxiv_stat_text"
os.makedirs(text_dir, exist_ok=True)

text_count = 0

for paper in os.listdir(pdf_dir):
    try:
        arxiv_id = paper
        pdf_path = os.path.join(pdf_dir, f"{arxiv_id}")
        text = extract_text_from_pdf(pdf_path)
        text_path = os.path.join(text_dir, f"{arxiv_id}.json")
        with open(text_path, "w") as f:
            json.dump({"text": text}, f)
        text_count += 1
    except Exception as e:
        print(f"Error processing {arxiv_id}: {e}")
print(f"Processed {text_count} papers")

Error processing 0903.3146v2.pdf: Failed to open file '/content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs/0903.3146v2.pdf'.
Error processing 2105.09095v3.pdf: Failed to open file '/content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs/2105.09095v3.pdf'.
Error processing 0812.3612v3.pdf: Failed to open file '/content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs/0812.3612v3.pdf'.
Error processing 1608.05122v3.pdf: Failed to open file '/content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs/1608.05122v3.pdf'.
Error processing 0706.0826v1.pdf: Failed to open file '/content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs/0706.0826v1.pdf'.
Error processing 0709.0871v1.pdf: Failed to open file '/content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs/0709.0871v1.pdf'.
Error processing 1604.01591v2.pdf: Failed to open file '/content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs/1604.01591v2.pdf'.
Error processing 1911.06361v3.pdf: Failed to open file '/conten